<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2

Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center> Тема 5. Композиции алгоритмов, случайный лес
## <center>Практика. Деревья решений и случайный лес в соревновании Kaggle Inclass по кредитному скорингу

Тут веб-формы для ответов нет, ориентируйтесь на рейтинг [соревнования](https://inclass.kaggle.com/c/beeline-credit-scoring-competition-2), [ссылка](https://www.kaggle.com/t/115237dd8c5e4092a219a0c12bf66fc6) для участия.

Решается задача кредитного скоринга. 

Признаки клиентов банка:
- Age - возраст (вещественный)
- Income - месячный доход (вещественный)
- BalanceToCreditLimit - отношение баланса на кредитной карте к лимиту по кредиту (вещественный)
- DIR - Debt-to-income Ratio (вещественный)
- NumLoans - число заемов и кредитных линий
- NumRealEstateLoans - число ипотек и заемов, связанных с недвижимостью (натуральное число)
- NumDependents - число членов семьи, которых содержит клиент, исключая самого клиента (натуральное число)
- Num30-59Delinquencies - число просрочек выплат по кредиту от 30 до 59 дней (натуральное число)
- Num60-89Delinquencies - число просрочек выплат по кредиту от 60 до 89 дней (натуральное число)
- Delinquent90 - были ли просрочки выплат по кредиту более 90 дней (бинарный) - имеется только в обучающей выборке

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

**Загружаем данные.**

In [2]:
train_df = pd.read_csv('../../../data/credit_scoring_train.csv', index_col='client_id')
test_df = pd.read_csv('../../../data/credit_scoring_test.csv', index_col='client_id')

In [3]:
train_df

,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit,Delinquent90
client_id,,,,,,,,,,
0,0.496289,49.1,13,0,0.0,2,0,5298.360639,0.387028,0
1,0.433567,48.0,9,2,2.0,1,0,6008.056256,0.234679,0
2,2206.731199,55.5,21,1,NaN,1,0,NaN,0.348227,0
3,886.132793,55.3,3,0,0.0,0,0,NaN,0.971930,0
4,0.000000,52.3,1,0,0.0,0,0,2504.613105,1.004350,0
...,...,...,...,...,...,...,...,...,...,...
74995,0.169463,67.6,5,1,0.0,0,0,20129.654428,0.074689,0
74996,2085.662215,43.2,1,1,0.0,0,0,NaN,1.005390,0
74997,0.118935,23.0,4,0,0.0,2,0,2019.117329,0.384511,0


In [4]:
y = train_df['Delinquent90']
train_df.drop('Delinquent90', axis=1, inplace=True)

In [5]:
train_df.head()

,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit
client_id,,,,,,,,,
0,0.496289,49.1,13,0,0.0,2,0,5298.360639,0.387028
1,0.433567,48.0,9,2,2.0,1,0,6008.056256,0.234679
2,2206.731199,55.5,21,1,NaN,1,0,NaN,0.348227
3,886.132793,55.3,3,0,0.0,0,0,NaN,0.971930
4,0.000000,52.3,1,0,0.0,0,0,2504.613105,1.004350


**Посмотрим на число пропусков в каждом признаке.**

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 0 to 74999
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DIR                    75000 non-null  float64
 1   Age                    75000 non-null  float64
 2   NumLoans               75000 non-null  int64  
 3   NumRealEstateLoans     75000 non-null  int64  
 4   NumDependents          73084 non-null  float64
 5   Num30-59Delinquencies  75000 non-null  int64  
 6   Num60-89Delinquencies  75000 non-null  int64  
 7   Income                 60153 non-null  float64
 8   BalanceToCreditLimit   75000 non-null  float64
dtypes: float64(5), int64(4)
memory usage: 5.7 MB


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 75000 to 149999
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DIR                    75000 non-null  float64
 1   Age                    75000 non-null  float64
 2   NumLoans               75000 non-null  int64  
 3   NumRealEstateLoans     75000 non-null  int64  
 4   NumDependents          72992 non-null  float64
 5   Num30-59Delinquencies  75000 non-null  int64  
 6   Num60-89Delinquencies  75000 non-null  int64  
 7   Income                 60116 non-null  float64
 8   BalanceToCreditLimit   75000 non-null  float64
dtypes: float64(5), int64(4)
memory usage: 5.7 MB


**Заменим пропуски медианными значениями.**

In [9]:
train_df['NumDependents'].fillna(train_df['NumDependents'].median(), inplace=True)
train_df['Income'].fillna(train_df['Income'].median(), inplace=True)
test_df['NumDependents'].fillna(test_df['NumDependents'].median(), inplace=True)
test_df['Income'].fillna(test_df['Income'].median(), inplace=True)

### Дерево решений без настройки параметров

**Обучите дерево решений максимальной глубины 3, используйте параметр random_state=17 для воспроизводимости результатов.**

In [10]:
first_tree = DecisionTreeClassifier(max_depth=3, random_state=17)
first_tree.fit(train_df, y)

DecisionTreeClassifier(max_depth=3, random_state=17)

**Сделайте прогноз для тестовой выборки.**

In [11]:
first_tree_pred = first_tree.predict_proba(test_df)[:, 1]

**Запишем прогноз в файл.**

In [12]:
def write_to_submission_file(predicted_labels, out_file,
                             target='Delinquent90', index_label="client_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(75000, 
                                                  predicted_labels.shape[0] + 75000),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [13]:
write_to_submission_file(first_tree_pred, 'credit_scoring_first_tree.csv')

**Если предсказывать вероятности дефолта для клиентов тестовой выборки, результат будет намного лучше.**

## Дерево решений с настройкой параметров с помощью GridSearch

**Настройте параметры дерева с помощью `GridSearhCV`, посмотрите на лучшую комбинацию параметров и среднее качество на 5-кратной кросс-валидации. Используйте параметр `random_state=17` (для воспроизводимости результатов), не забывайте про распараллеливание (`n_jobs=-1`).**

In [22]:
tree_params = {'max_depth': list(range(3, 8)), 
               'min_samples_leaf': list(range(5, 20))}

locally_best_tree = GridSearchCV(DecisionTreeClassifier(random_state=17), param_grid=tree_params, n_jobs=-1, \
                                 scoring='roc_auc', cv=5)
locally_best_tree.fit(train_df, y)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=17), n_jobs=-1,
             param_grid={'max_depth': [3, 4, 5, 6, 7],
                         'min_samples_leaf': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                                              15, 16, 17, 18, 19]},
             scoring='roc_auc')

In [23]:
locally_best_tree.best_params_, round(locally_best_tree.best_score_, 3)

({'max_depth': 6, 'min_samples_leaf': 15}, 0.83)

**Сделайте прогноз для тестовой выборки и пошлите решение на Kaggle.**

In [24]:
tuned_tree_pred_probs = locally_best_tree.predict_proba(test_df)[:, 1]

In [25]:
write_to_submission_file(tuned_tree_pred_probs, 'tuned_tree.csv')

### Случайный лес без настройки параметров

**Обучите случайный лес из деревьев неограниченной глубины, используйте параметр `random_state=17` для воспроизводимости результатов.**

In [18]:
first_forest = RandomForestClassifier(random_state=17)
first_forest.fit(train_df, y)

RandomForestClassifier(random_state=17)

In [19]:
first_forest_pred = first_forest.predict_proba(test_df)[:, 1]

**Сделайте прогноз для тестовой выборки и пошлите решение на Kaggle.**

In [20]:
write_to_submission_file(first_forest_pred, 'forest_firest.csv')

In [21]:
train_df.shape

(75000, 9)

### Случайный лес c настройкой параметров

**Настройте параметр `max_features` леса с помощью `GridSearhCV`, посмотрите на лучшую комбинацию параметров и среднее качество на 5-кратной кросс-валидации. Используйте параметр random_state=17 (для воспроизводимости результатов), не забывайте про распараллеливание (n_jobs=-1).**

In [35]:
%%time
forest_params = {'max_depth': [5, 40, 5],
                 'min_samples_leaf': [2, 4, 6, 8, 12, 15]}

locally_best_forest = GridSearchCV(RandomForestClassifier(random_state=17, max_features='sqrt'), param_grid=forest_params, \
                                   cv=5, n_jobs=-1, scoring='roc_auc', verbose=3)
locally_best_forest.fit(train_df, y)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
CPU times: user 10.5 s, sys: 348 ms, total: 10.8 s
Wall time: 3min 26s


GridSearchCV(cv=5,
             estimator=RandomForestClassifier(max_features='sqrt',
                                              random_state=17),
             n_jobs=-1,
             param_grid={'max_depth': [5, 40, 5],
                         'min_samples_leaf': [2, 4, 6, 8, 12, 15]},
             scoring='roc_auc', verbose=3)

In [36]:
locally_best_forest.best_params_, round(locally_best_forest.best_score_, 3)

({'max_depth': 40, 'min_samples_leaf': 15}, 0.844)

In [37]:
tuned_forest_pred = locally_best_forest.predict_proba(test_df)[:, 1]

In [38]:
write_to_submission_file(tuned_forest_pred, 'tuned_forest.csv')

**Посмотрите, как настроенный случайный лес оценивает важность признаков по их влиянию на целевой. Представьте результаты в наглядном виде с помощью `DataFrame`.**

In [39]:
pd.DataFrame(locally_best_forest.best_estimator_.feature_importances_, index=train_df.columns, columns=['coef']) \
        .sort_values(by='coef', ascending=False)

,coef
BalanceToCreditLimit,0.285093
Num60-89Delinquencies,0.194688
Num30-59Delinquencies,0.140417
Age,0.099023
DIR,0.094657
Income,0.084577
NumLoans,0.061758
NumRealEstateLoans,0.020311
NumDependents,0.019476


[CV 4/5] END ...max_depth=5, min_samples_leaf=2;, score=0.833 total time=   5.4s
[CV 5/5] END ...max_depth=5, min_samples_leaf=2;, score=0.838 total time=   5.4s
[CV 4/5] END ...max_depth=5, min_samples_leaf=4;, score=0.833 total time=   5.8s
[CV 3/5] END ...max_depth=5, min_samples_leaf=6;, score=0.848 total time=   6.0s
[CV 2/5] END ...max_depth=5, min_samples_leaf=8;, score=0.849 total time=   5.8s
[CV 1/5] END ..max_depth=5, min_samples_leaf=12;, score=0.837 total time=   6.0s
[CV 5/5] END ..max_depth=5, min_samples_leaf=12;, score=0.839 total time=   5.8s
[CV 4/5] END ..max_depth=5, min_samples_leaf=15;, score=0.833 total time=   5.7s
[CV 3/5] END ..max_depth=40, min_samples_leaf=2;, score=0.832 total time=  14.7s
[CV 2/5] END ..max_depth=40, min_samples_leaf=4;, score=0.842 total time=  14.3s
[CV 1/5] END ..max_depth=40, min_samples_leaf=6;, score=0.841 total time=  14.9s
[CV 5/5] END ..max_depth=40, min_samples_leaf=6;, score=0.836 total time=  15.7s
[CV 4/5] END ..max_depth=40,

**Обычно увеличение количества деревьев только улучшает результат. Так что напоследок обучите случайный лес из 300 деревьев с найденными лучшими параметрами. Это может занять несколько минут.**

In [41]:
%%time
final_forest = RandomForestClassifier(n_jobs=-1, n_estimators=500, random_state=17, max_features=3, max_depth=40, min_samples_leaf=15)
final_forest.fit(train_df, y)
final_forest_pred = final_forest.predict_proba(test_df)[:, 1]
write_to_submission_file(final_forest_pred, 'credit_scoring_final_forest.csv')

CPU times: user 1min 25s, sys: 469 ms, total: 1min 26s
Wall time: 23.2 s


**Сделайте посылку на Kaggle.**